In [1]:
import os
import time
import random
from datetime import datetime
from itertools import islice
from collections import defaultdict, namedtuple
from tuplestate import init_from_solvitaire, to_dict, pprint_st
from benchmarking import *
from solver import solve

In [2]:
def get_state(ret):
    deck_json = convert_shootme_to_solvitaire_json(ret)
    return init_from_solvitaire(deck_json)


def prepare_ui_state(ret):
    parsed = parse_winnable(ret)
    game_state = to_dict(get_state(ret))
    
    def card(c):
        ''' UI uses 10 instead of T '''
        return c.replace('t', '10').replace('T', '10')
    
    return {
        'stock': [card(c).lower() for c in game_state['stock']],
        'tableau': [
            [card(c) for c in tab] 
            for tab in game_state['tableau']
        ],
        'waste': game_state['waste'],
        'foundation': game_state['foundation'],
        'moveSeq': parsed['moves'].strip().split(' ')
    }

In [3]:
results = defaultdict(set)

all_solutions = os.listdir('./bench/shootme/')
seeds = list(map(lambda fname: int(fname[:-4]), all_solutions))

def solve_state(ret):
    lines = ret.splitlines()
    result = lines[15]
    if result.startswith('Minimal solution'):
        return "Solved-Min"
    elif result.startswith("Solved"):
        return "Solved"
    elif result.startswith('Impossible'):
        return "Impossible"
    elif result.startswith('Unknown'):
        return "Unknown"

for seed in sorted(seeds):
    with open(f"bench/shootme/{seed}.txt") as f:
        ret = f.read()
        result = solve_state(ret)
        results[result].add(seed)
        
for res, seeds in results.items():
    print(f"{res:12} {len(seeds):8,}")
total = sum(len(s) for s in results.values())
print(('-'*12) + '-' + ('-'*8))
print(f"{'Total':12} {total:8,}")

Solved-Min      7,876
Impossible      1,282
Unknown           522
Solved            320
---------------------
Total          10,000


In [4]:
seeds = list(range(1, 10_001))

In [5]:
suite1 = random.sample(seeds, k=10)

In [6]:
suite1

[2599, 8677, 1011, 8635, 8394, 2886, 8867, 4605, 2286, 7123]

In [7]:
def map_seeds_to_states(seed_seq):
    for seed in seed_seq:
        with open(f"bench/shootme/{seed}.txt") as f:
            ret = f.read()
            state = get_state(ret)
            yield seed, state

In [8]:
Result = namedtuple('Result', ['seed', 'time', 'seq', 'seqlen', 'datetime'])

In [9]:
solvemin = list(islice(map_seeds_to_states(results['Solved-Min']), 3))
seed, state = solvemin[0]
state

KlonState(stock=('TH', '8D', '7H', 'KC', '3D', 'TD', '5H', '8C', 'QS', 'JS', '9H', '7C', '6D', '8S', 'QD', 'QH', 'AC', '9C', '5S', 'KH', 'QC', '2S', 'TC', '9D'), tableau1=('4D',), tableau2=('3c', '4C'), tableau3=('9s', '7d', '6H'), tableau4=('ah', '7s', '2c', 'JD'), tableau5=('ad', '6s', '2h', 'ts', 'AS'), tableau6=('4h', '8h', '2d', 'ks', 'jc', '5C'), tableau7=('3h', '4s', '3s', '5d', 'jh', '6c', 'KD'), waste=(), foundation1=(), foundation2=(), foundation3=(), foundation4=())

In [10]:
def run_solver(seed, state):
    start = time.time()
    moveseq = solve(state)
    seq = " ".join(moveseq)
    end = time.time()
    elapsed = end-start
    now = datetime.now().isoformat()
    return Result(seed=seed, time=elapsed, seq=seq, seqlen=len(moveseq), datetime=now)

In [11]:
ret = run_solver(*solvemin[1])
ret

Result(seed=2, time=1.124732494354248, seq='5C 6H 67 12 31 63 32-2 DR1 DR1 DR1 DR1 DR1 W6 DR1 DR1 DR1 W5 W2 72-2 73 7S 72 7H 47 46 DR1 DR1 DR1 W3 DR1 DR1 W5 DR1 W4 DR3 DR1 W1 71-2 72 DR1 W7 27-9 DR1 DR1 W5 DR1 DR1 DR1 W6 DR1 DR1 DR1 DR1 DR1 DR1 12 DR1 DR1 DR1 DR1 73-5 DR1 DR1 DR1 DR1 21 DR1 DR1 DR1 DR1 54-2 DR1 DR1 DR1 DR1 12 DR1 DR1 DR1 DR1 37-5 DR1 DR1 DR1 DR1 21 DR1 DR1 DR1 DR1 WD W4 W3 63-4 56-2 52 5C 61-3 61 W6 W2 W3 42-5 4H 51 76-9 DR1 W4 W1 W1 DR1 W2 W4 W7 2C 6C 24-8 37-9 DR1 W1 W7 1D 7S 7D 4D 4C 1S 1H 6H 6C 7S 7D 4D 4C 1S 1H 6H 6C 4H 7S 7D 1S 1D 1C 6D 7S 7H 1H 7C 4S 4D 7D 6S 6H 4C 6C 1S 1D 4H 6D 4C 7S 1S 7H', seqlen=164, datetime='2019-11-25T17:41:24.558499')